In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
HOME = '/srv/home/christinedk/wp_internship/'
DATA_DIR = HOME + 'data/'
sys.path.append(HOME + 'collaboration/')

In [3]:
import pandas as pd
import json
import numpy as np
from collections import Counter

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler

from math import log2
from utils import entropy

/usr/lib/python3/dist-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/usr/lib/python3/dist-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, copy_X=True, fit_path=True,
/usr/lib/python3/dist

In [4]:
# Page history

In [5]:
template='advert'

In [6]:
with open(HOME+'features/activity_{}.json'.format(template),'rb') as f:
    features_pos = json.load(f)
with open(HOME+'negative_features/activity_{}.json'.format(template),'rb') as f:
    features_neg = json.load(f)

In [7]:
len(features_pos)

7570

In [8]:
len(features_neg)

6045

In [9]:
article_features_neg = pd.DataFrame([d['article'] for d in features_neg])
article_features_neg['label'] = 0
article_features_pos = pd.DataFrame([d['article'] for d in features_pos])
article_features_pos['label'] = 1

In [10]:
article_data = pd.concat([article_features_neg, article_features_pos])
labels = article_data['label']
feat = article_data.drop('label',axis=1)

In [11]:
len(article_data)

13615

In [12]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
8,frac_recent_revisions,0.305192,0.305192
11,concentration_ratio,0.133609,0.133609
14,recent_response_time,0.125896,0.125896
13,recent_edit_size,0.108532,0.108532
12,contribution_frac_entropy,-0.098823,0.098823
0,edit_size,0.094759,0.094759
1,time_to_respond,0.083943,0.083943
9,top_contributor_frac,0.077974,0.077974
6,num_revisions,-0.046778,0.046778
10,frac_anon_revisions,-0.046518,0.046518


In [13]:
# linear model

In [14]:
hypers = {'solver':'lbfgs','max_iter':10000,'C':10000}

In [15]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/lib/python3/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/lib/python3/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/christinedk/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys


In [16]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.7316068312644818

In [17]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[1040,  443],
       [ 724, 1197]])

In [18]:
f1_score(y_pred=y_pred, y_true=y_test)

0.6722830665543387

In [19]:
# user-article

In [20]:
def aggregate_features(article_users):
    if len(article_users) == 0:
        return {}
    else:
        article_users = pd.DataFrame(article_users).drop('event_user_id',axis=1)
        means = {'mean_'+key:val for key, val in article_users.mean().items()}
        std = {'std_'+key:val for key, val in article_users.std().items()}
        max_ = {'max_'+key:val for key, val in article_users.max().items()}
        ent = {'frac_page_edits_ent': entropy(article_users.frac_page_edits)}
        features = {**means, **std, **max_, **ent}

        return features

In [21]:
user_article_features_pos = pd.DataFrame([aggregate_features(d['user_article']) 
                                          for d in features_pos])
user_article_features_neg = pd.DataFrame([aggregate_features(d['user_article']) 
                                          for d in features_neg])

In [22]:
user_article_features_pos['label'] = 1
user_article_features_neg['label'] = 0

user_article_data = pd.concat([user_article_features_neg,user_article_features_pos])
labels = user_article_data['label']
feat = user_article_data.drop('label',axis=1)

In [23]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
5,mean_frac_page_edits,0.216113,0.216113
24,frac_page_edits_ent,-0.113989,0.113989
2,mean_time_to_respond,0.086815,0.086815
21,max_frac_page_edits,0.085800,0.085800
0,mean_edit_size,0.084020,0.084020
13,std_frac_page_edits,0.080455,0.080455
3,mean_time_responded_to,0.079513,0.079513
10,std_time_to_respond,0.066997,0.066997
11,std_time_responded_to,0.056178,0.056178
20,max_num_edits,-0.052788,0.052788


In [24]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [25]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.6397163865546218

In [26]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[ 845,  655],
       [ 739, 1165]])

In [27]:
f1_score(y_pred=y_pred, y_true=y_test)

0.6256713211600429

In [28]:
# talk vol

In [29]:
with open(HOME+'features/talk_{}.json'.format(template),'rb') as f:
    talk_pos = json.load(f)
with open(HOME+'negative_features/talk_{}.json'.format(template),'rb') as f:
    talk_neg = json.load(f)

In [30]:
talk_vol_neg = pd.DataFrame([d['talk_volume'] for d in talk_neg])
talk_vol_neg['label'] = 0
talk_vol_pos = pd.DataFrame([d['talk_volume'] for d in talk_pos])
talk_vol_pos['label'] = 1

In [31]:
talk_vol = pd.concat([talk_vol_neg,talk_vol_pos])
labels = talk_vol['label']
feat = talk_vol.drop('label',axis=1)

In [32]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
5,mean_response_time,0.064857,0.064857
1,frac_recent_revisions,0.030878,0.030878
2,num_editors,-0.026086,0.026086
4,mean_edit_size,-0.025699,0.025699
0,num_revisions,-0.021082,0.021082
6,page_talk_ratio,-0.019187,0.019187
3,top_contributor_frac,0.012608,0.012608


In [33]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/lib/python3/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/lib/python3/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/christinedk/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys


In [34]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.549895324250742

In [35]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[ 196, 1336],
       [ 174, 1698]])

In [36]:
f1_score(y_pred=y_pred, y_true=y_test)

0.6922136159804322

In [37]:
# talk lang

In [38]:
with open(HOME+'features/talk_{}.json'.format(template),'rb') as f:
    talk_pos = json.load(f)
with open(HOME+'negative_features/talk_{}.json'.format(template),'rb') as f:
    talk_neg = json.load(f)

In [39]:
def aggregate_language(talk_feat):
    if len(talk_feat) == 0:
        return {}
    else:
        all_utts = pd.DataFrame(list(np.concatenate(list(talk_feat.values()))))
        means = {'mean_'+key:val for key, val in all_utts.mean().items()}
        std = {'std_'+key:val for key, val in all_utts.std().items()}
        max_ = {'max_'+key:val for key, val in all_utts.max().items()}
        features = {**means, **std, **max_}

        return features

In [40]:
talk_lang_neg = pd.DataFrame([aggregate_language(d['talk_language']) for d in talk_neg])
talk_lang_neg['label'] = 0

In [41]:
len(talk_lang_neg)

6045

In [42]:
talk_lang_pos = pd.DataFrame([aggregate_language(d['talk_language']) for d in talk_pos])
talk_lang_pos['label'] = 1

In [43]:
len(talk_lang_pos)

7570

In [44]:
talk_lang = pd.concat([talk_lang_neg,talk_lang_pos])
labels = talk_lang['label']
feat = talk_lang.drop('label',axis=1)

In [45]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
65,max_reply_depth,-0.033367,0.033367
51,max_politeness_markers_==Gratitude==,-0.031908,0.031908
55,max_politeness_markers_==1st_person_start==,-0.030567,0.030567
44,max_politeness_markers_==Please==,-0.029829,0.029829
21,mean_reply_depth,-0.029532,0.029532
46,max_politeness_markers_==HASHEDGE==,-0.029158,0.029158
56,max_politeness_markers_==2nd_person==,-0.028984,0.028984
54,max_politeness_markers_==1st_person==,-0.028632,0.028632
45,max_politeness_markers_==Please_start==,-0.027598,0.027598
48,max_politeness_markers_==Hedges==,-0.026818,0.026818


In [46]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)
#C=0.1,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [47]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.48331837758969587

In [48]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[  76, 1406],
       [  88, 1834]])

In [49]:
f1_score(y_pred=y_pred, y_true=y_test)

0.710577295621852

In [50]:
# combined

In [51]:
labels = article_data['label']

feat = pd.concat([ds.reset_index(drop=True).drop('label',axis=1) 
                  for ds in [article_data,talk_vol,talk_lang,user_article_data]],axis=1) #editor_data,collab_data

In [52]:
len(feat)

13615

In [53]:
correlations = feat.corrwith(labels).reset_index().rename(columns={'index':'feature',0:'score'})
correlations['abs'] = correlations.score.abs()
correlations.sort_values(by='abs', ascending=False)[:10]

,feature,score,abs
2,article_age_years,0.068934,0.068934
12,contribution_frac_entropy,0.065030,0.065030
10,frac_anon_revisions,0.063187,0.063187
109,max_frac_page_edits,-0.062240,0.062240
101,std_frac_page_edits,-0.056771,0.056771
9,top_contributor_frac,-0.056593,0.056593
14,recent_response_time,0.055798,0.055798
16,frac_recent_revisions,-0.051686,0.051686
1,time_to_respond,0.048907,0.048907
112,frac_page_edits_ent,0.044784,0.044784


In [54]:
scaler = StandardScaler()
model = LogisticRegression(**hypers)#,class_weight={0:1,1:5}

X_train, X_test, y_train, y_test = train_test_split(feat.fillna(0),labels)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/lib/python3/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/lib/python3/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/christinedk/venv/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [55]:
model.fit(X=X_train,y=y_train)
predictions=model.predict_proba(X_test)
y_pred = np.argmax(predictions,axis=1)
roc_auc_score(y_score=predictions[:,1],y_true=y_test)

0.7230931550951848

In [56]:
confusion_matrix(y_pred=y_pred,y_true=y_test)

array([[ 996,  508],
       [ 684, 1216]])

In [57]:
f1_score(y_pred=y_pred, y_true=y_test)

0.6710816777041944

In [178]:
# Thoughts:

# article:
# comparing to same pages means some features are less relevant; eg. num_editors, mean edit_size, top_contributor_frac, etc
# recency features should be more important 
#     - calculate all features for recent revisions only as well?


# user-article
# empty for some negatives? sample page': 189559,'date': '2003-02-28 16:09:01'
# revert features are incorrect, need to re-export

In [ ]:
# Editor